In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from rosbags.rosbag2 import Reader
from rosbags.serde import deserialize_cdr
from rosbags.highlevel import AnyReader

import numpy as np

In [ ]:
def get_pose_data(bag_path, topic_name):
    poses_data = {
        'timestamp': [],
        'time_sec': [],
        'position_x': [],
        'position_y': [],
        'position_z': [],
        'orientation_x': [],
        'orientation_y': [],
        'orientation_z': [],
        'orientation_w': []
    }

    with AnyReader([bag_path]) as reader:
        # Print bag info
        print(f"Bag duration: {reader.duration / 1e9:.2f} seconds")
        print(f"Message count: {reader.message_count}")

        connection = [x for x in reader.connections if x.topic == topic_name]

        # Read messages
        for connection, timestamp, rawdata in reader.messages(connections=connection):
            msg = reader.deserialize(rawdata, connection.msgtype)

            # Extract pose data
            poses_data['timestamp'].append(timestamp)
            poses_data['time_sec'].append(timestamp / 1e9)

            poses_data['position_x'].append(msg.position.x)
            poses_data['position_y'].append(msg.position.y)
            poses_data['position_z'].append(msg.position.z)

            poses_data['orientation_x'].append(msg.orientation.x)
            poses_data['orientation_y'].append(msg.orientation.y)
            poses_data['orientation_z'].append(msg.orientation.z)
            poses_data['orientation_w'].append(msg.orientation.w)

    poses_data = pd.DataFrame(poses_data)
    
    # clean up time_sec
    min_timestamp = poses_data['time_sec'][0]
    poses_data['time_sec'] = poses_data['time_sec'] - min_timestamp
    return poses_data

def get_cmd_vel_data(bag_path, topic_name):
    cmd_vel_data = {
        'timestamp': [],
        'time_sec': [],
        'linear_x': [],
        'angular_z': []
    }

    with AnyReader([bag_path]) as reader:
        # Print bag info
        print(f"Bag duration: {reader.duration / 1e9:.2f} seconds")
        print(f"Message count: {reader.message_count}")

        connection = [x for x in reader.connections if x.topic == topic_name]

        # Read messages
        for connection, timestamp, rawdata in reader.messages(connections=connection):
            msg = reader.deserialize(rawdata, connection.msgtype)

            # Extract pose data
            cmd_vel_data['timestamp'].append(timestamp)
            cmd_vel_data['time_sec'].append(timestamp / 1e9)
            cmd_vel_data['linear_x'].append(msg.linear.x)
            cmd_vel_data['angular_z'].append(msg.angular.z)

    return cmd_vel_data

def get_force_data(bag_path, topic_name):
    force_data = {
        'timestamp': [],
        'time_sec': [],
        'data': []
    }
    with AnyReader([bag_path]) as reader:
        # Print bag info
        print(f"Bag duration: {reader.duration / 1e9:.2f} seconds")
        print(f"Message count: {reader.message_count}")

        connection = [x for x in reader.connections if x.topic == topic_name]

        # Read messages
        for connection, timestamp, rawdata in reader.messages(connections=connection):
            msg = reader.deserialize(rawdata, connection.msgtype)
            force_data['timestamp'].append(timestamp)
            force_data['time_sec'].append(timestamp / 1e9)
            force_data['data'].append(msg.data)

    return force_data


def plot_displacements_over_time(timestamps, monica_poses, ross_poses):
    displacements = monica_poses['position_x'].values - ross_poses['position_x'].values

    plt.plot(
        timestamps,
        displacements,
        '-r'
    )
    plt.legend(['monica', 'ross'])
    plt.grid(True)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Trajectory over time for monica and ross')
    plt.show()

def plot_force_data(timestamps, force_data):
    plt.plot(
        timestamps,
        force_data,
        '-r'
    )
    plt.grid(True)
    plt.xlabel('t')
    plt.ylabel('force (N)')
    plt.title('Force readings over time')
    plt.show()


def clean_data(monica_data, ross_data, monica_cmd_vel_data, ross_cmd_vel_data):
    # extract the time the robots started moving and then stopped
    monica_cmd_vel_data.drop(monica_cmd_vel_data[monica_cmd_vel_data['linear_x'] < 0])
    ross_cmd_vel_data.drop(ross_cmd_vel_data[ross_cmd_vel_data['linear_x'] < 0])

    start_timestamp = (monica_cmd_vel_data['time_sec'][0] + ross_cmd_vel_data['time_sec'][0]) / 2.0
    end_timestamp = (monica_cmd_vel_data['time_sec'][-1] + ross_cmd_vel_data['time_sec'][-1]) / 2.0
    
    # remove all values outside start_timestamp and end_timestamp
    monica_data = monica_data[monica_data['time_sec'] >= start_timestamp | monica_data[monica_data] <= end_timestamp]
    ross_data = ross_data[ross_data['time_sec'] >= start_timestamp | ross_data[monica_data] <= end_timestamp]
    
    # if the number of timestamps is different
    # drop a few in the beginning
    num_of_timestamps_monica = monica_data['time_sec'].values.shape[0]
    num_of_timestamps_ross = ross_data['time_sec'].values.shape[0]

    if num_of_timestamps_monica > num_of_timestamps_ross:
        diff = num_of_timestamps_monica - num_of_timestamps_ross
        ross_data = ross_data.iloc[diff:].reset_index(drop=True)

    elif num_of_timestamps_ross > num_of_timestamps_monica:
        diff = num_of_timestamps_ross - num_of_timestamps_monica
        monica_data = monica_data.iloc[diff:].reset_index(drop=True)

    
     

In [ ]:
bag_path = Path('../../../datasets/vicon_data/vicon_test_1')

ross_poses = get_pose_data(bag_path, '/world_ross_pose')
monica_poses = get_pose_data(bag_path, '/world_monica_pose')


Bag duration: 49.49 seconds
Message count: 101001
Bag duration: 49.49 seconds
Message count: 101001


0.0